1. Intall libraries: pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
2. Set up your Google Cloud Environment:
    - Go to: console.cloud.google.com
    - Create a new project
    - Create a new API: Go to APIs & Services -> Enable APIs & Services
    - Search for GMAIL API and enable it
    - Enable OAUTH: Go to APIs & Services -> OAuth consent screen
        - Start a new channel
        - Name your app
        - Select EXTERNAL 
        - Create the OAuth
        - Go to PUBLIC (left panel) and add test users
    - Create credentials:
        - New Credential -> OAUTH client ID
        - App type: Desktop
        - Name your app and create it
        - Once your app credentials are created, DOWNLOAD the JSON file


In [3]:
import os
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

In [5]:
# REFERENCE: https://github.com/BMMR/pythonBolt/blob/54428902c54fd8b01f7fc3e00a39145a72413da3/google_apis.py
def create_service(client_secret_file, api_name, api_version, *scopes, prefix=''):
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]

    creds = None
    working_dir = os.getcwd()
    token_dir = 'token files'
    token_file = f'token_{API_SERVICE_NAME}_{API_VERSION}{prefix}.json'

    ### Check if token dir exists first, if not, create the folder
    if not os.path.exists(os.path.join(working_dir, token_dir)):
        os.mkdir(os.path.join(working_dir, token_dir))

    if os.path.exists(os.path.join(working_dir, token_dir, token_file)):
        creds = Credentials.from_authorized_user_file(os.path.join(working_dir, token_dir, token_file), SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(os.path.join(working_dir, token_dir, token_file), 'w') as token:
            token.write(creds.to_json())

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=creds, static_discovery=False)
        print(API_SERVICE_NAME, API_VERSION, 'service created successfully')
        return service
    except Exception as e:
        print(e)
        print(f'Failed to create service instance for {API_SERVICE_NAME}')
        os.remove(os.path.join(working_dir, token_dir, token_file))
        return None

In [6]:
client_secret_file = 'client_secret.json'
API_SERVICE_NAME = 'gmail'
API_VERSION = 'v1'
SCOPES =['https://mail.google.com/']
service = create_service(client_secret_file, API_SERVICE_NAME, API_VERSION, SCOPES)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=478639231328-tntgmc6q72bo1v5psvhsfd0dmd9sf9lp.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50932%2F&scope=https%3A%2F%2Fmail.google.com%2F&state=T8jAXnjnM3rTIedMCUoQScpZHD7FaF&access_type=offline
gmail v1 service created successfully


In [7]:
import os
import base64
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

In [10]:
def send_email(service, to, subject, body, body_type='plain'):
    message = MIMEMultipart()
    message['to'] = to
    message['cc'] = 'ctoruno@worldjusticeproject.org'
    message['subject'] = subject

    if body_type.lower() not in ['plain', 'html']:
        raise ValueError('body_type must be either "plain" or "html"')

    message.attach(MIMEText(body, body_type.lower()))

    raw_message = base64.urlsafe_b64encode(message.as_bytes()).decode('utf-8')

    sent_message = service.users().messages().send(
        userId = 'me', 
        body   = {'raw': raw_message}
    ).execute()

    return sent_message

In [11]:
recipient = 'dgranada@equilibriumbdc.com'
email_subject = 'TESTING GMAIL API'
email_body = 'TESTING MAIL API'

In [12]:
response_email = send_email(service, recipient, email_subject, email_body)
print(response_email)

{'id': '1953870c7155f7cc', 'threadId': '1953870c7155f7cc', 'labelIds': ['SENT']}
